In [1]:
import tensorflow as tf

基本創造tensor的範例

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


發現他不會真的印出值，因為他們要在被evaluate的時候，才會產生值。

而要evaluate這些tensor，要在__session__當中跑這個computational graph。

要產生一個session並run一個computational graph，程式碼如下：

In [7]:
with tf.Session() as sess:
    print(sess.run([node1, node2]))

[3.0, 4.0]


可以透過tf.add做tensor的加法

In [9]:
sess = tf.Session()
node3 = tf.add(node1, node2)
print(node3)
print(sess.run(node3))

Tensor("Add:0", shape=(), dtype=float32)
7.0


__placeholder__用來裝外來資料，先宣告，之後要倒進去dict型態

In [10]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + 等於 tf.add()

In [11]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


可以更複雜一點

In [12]:
add_and_triple = adder_node * 3
sess.run(add_and_triple, {a:3, b:4.5})

22.5

機器學習當中，我們常常要一些變數，那些變數是可以被初始化，也可以被改變（例如訓練過程更新權重），則可以使用__Variable__。

In [13]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

初始化全部的__Variable__

In [14]:
init = tf.global_variables_initializer()
sess.run(init)

線性模型的輸出即是

In [15]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


機器學習的第一步「定義模型」已經完成，接下來就是要定義衡量模型好壞的方法，也就是evaluation，也就是我們要定義我們的__loss funtion__。

這邊我們使用線性回歸模型常用的loss function - MSE

In [17]:
y = tf.placeholder(tf.float32)
# 以下兩行即是MSE的算法
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


__tf.train API__，計算gradient並更新權重

In [19]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [23]:
sess = tf.Session()
sess.run(init)
for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

訓練結果的W和b

In [26]:
print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


整個完整的「linear regression」程式如下：

In [28]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


__tf.estimator__是一個high level的TensorFlow函式，他簡化機器學習的過程，其包含以下步驟：

1) 執行training loop(estimator.train)

2) 執行evaluation loop(estimator.evaluate)

3) 管理dataset(estimator.input)

(estimator也有內建許多常用的model，例如__estimator.LinearRegressor__)

剛剛的linear regression使用__tf.estimator__將會如以下簡單：

In [29]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# 宣告一個一個feature list，這邊我們只有一個維度的numeric feature，tf.feature_column
# 另外還有許多更複雜且好用的feature_column
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# estimator是一個API，用來執行training(fitting)和evaluation(inference)，首先要先
# 定義estimator的類型，這邊有許多內建的模型，例如linear regression, linear classification
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# tf.estimator.inputs提供許多函式可以讀取和設定data sets，這邊我們使用numpy_input_fn
# 來設定我們的兩個data sets: 一個用於training，一個用於evaluation
# input_fn中我們需要告訴函式我們要江資料分成多少個batches，和一個batch要多大
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# 內建的estimator.LinearRegressor已經設定Loss Function就是MSE，optimizer就是用
# gradient descent，所以這邊只需要用estimator.train函式就可以進行訓練，
# 不用使用迴圈跑sess.run來更新W
estimator.train(input_fn=input_fn, steps=1000)

# 我們也可以用estimator.evaluate來分別evaluate traing set的loss，和testing set的loss
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpokyakify', '_service': None, '_tf_random_seed': None, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_task_type': 'worker', '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4954e08ef0>, '_log_step_count_steps': 100, '_session_config': None}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpokyakify', '_service': None, '_tf_random_seed': None, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_task_type': 'worker', '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4954e08ef0>, '_log_step_count_steps': 100, '_session_config': None}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2017-11-22-06:41:00


INFO:tensorflow:Finished evaluation at 2017-11-22-06:41:00


INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00252574, global_step = 1000, loss = 0.0101029


INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00252574, global_step = 1000, loss = 0.0101029


train metrics: {'global_step': 1000, 'average_loss': 6.5057293e-11, 'loss': 2.6022917e-10}
eval metrics: {'global_step': 1000, 'average_loss': 0.0025257356, 'loss': 0.010102943}


__tf.estimator__也不限制於內建的model，我們也可以自己定義model，只要使用estimator.Estimator

事實上，estimator.LinearRegressor也只是estimator.Estimator的一個sub-class，

自定義model的方式只要給Estimator一個function __model_fn__，其中告訴tf.estimator模型的function set，

它的loss function以及optimize的方式即可（就跟自幹一個linear regression一樣，只是寫在model_fn裡）

如以下程式：

In [30]:
import numpy as np
import tensorflow as tf

# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmptsq5qrmp', '_service': None, '_tf_random_seed': None, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_task_type': 'worker', '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f48c03bc320>, '_log_step_count_steps': 100, '_session_config': None}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmptsq5qrmp', '_service': None, '_tf_random_seed': None, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_task_type': 'worker', '_save_checkpoints_steps': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f48c03bc320>, '_log_step_count_steps': 100, '_session_config': None}


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Finished evaluation at 2017-11-22-06:51:15


INFO:tensorflow:Finished evaluation at 2017-11-22-06:51:15


INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101007


INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101007


train metrics: {'global_step': 1000, 'loss': 1.4822978e-11}
eval metrics: {'global_step': 1000, 'loss': 0.010100724}
